In [1]:
import time
import pandas as pd
import datetime
import os
import itertools
import calendar

In [2]:
# 檔案讀取位置
datasite = './TimeSeriesData/TS6-201902_202306.parquet'

# 是否輸出檔案 >0 輸出
df_save = 1
# 檔案輸出位置 
custom_output_path = f'./TimeSeriesData'
DataName = 'TS10-test'


# 设置要補滿的起始年份、月份和结束年份、月份(多設定一個月forMA，最後再排除)
start_year = 2023
start_month = 9
end_year = 2023
end_month = 12

#輸出月份
output_start_day_str = '2023-09-20'  #含這天
output_end_day_str   = '2024-01-01'  #不含這天

#-------------------------------------------------------------------------------------------------#
output_start_day = datetime.datetime.strptime(output_start_day_str, '%Y-%m-%d')
output_end_day = datetime.datetime.strptime(output_end_day_str, '%Y-%m-%d')

# 記錄開始時間
start_time = time.time()

df = pd.read_parquet(datasite)

# 記錄結束時間
end_time = time.time()   
# 計算程式執行花費的時間
execution_time = end_time - start_time

print("程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())
df

程式執行花費的時間： 2.37 秒
done時間: 2023-09-14 17:22:12.494987


,Name,year,month,day,hour,PULocationID,weekday,is_holiday,count,lat,lon,countN
0,lyft,2019,2,1,0,3,4,False,4,40.864294,-73.846510,4.0
1,lyft,2019,2,1,1,3,4,False,2,40.864294,-73.846510,2.0
2,lyft,2019,2,1,2,3,4,False,6,40.864294,-73.846510,6.0
3,lyft,2019,2,1,3,3,4,False,1,40.864294,-73.846510,1.0
4,lyft,2019,2,1,4,3,4,False,1,40.864294,-73.846510,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
30273907,yellow,2023,6,30,19,199,4,False,0,40.793489,-73.880683,0.0
30273908,yellow,2023,6,30,20,199,4,False,0,40.793489,-73.880683,0.0
30273909,yellow,2023,6,30,21,199,4,False,0,40.793489,-73.880683,0.0
30273910,yellow,2023,6,30,22,199,4,False,0,40.793489,-73.880683,0.0


In [3]:
# 記錄開始時間
start_time = time.time()

# 创建所有可能的组合
years = range(start_year, end_year + 1)
months = range(1,13)
location_ids = df['PULocationID'].unique()  # 用您的位置ID列表替换
names = range(1,4)
print('地點數量:',df['PULocationID'].nunique())

combinations = []

for name,year, month, location_id in itertools.product(names,years, months, location_ids):
    if start_year == end_year :
        if start_month <= month <= end_month:
            days_in_month = calendar.monthrange(year, month)[1]
            days = range(1, days_in_month + 1)
            hours = range(24)
            for day, hour in itertools.product(days, hours):
                combinations.append((name,year, month, day, hour, location_id))
    elif  start_year < end_year:
        if year == start_year:
            if month >= start_month:
                days_in_month = calendar.monthrange(year, month)[1]
                days = range(1, days_in_month + 1)
                hours = range(24)
                for day, hour in itertools.product(days, hours):
                    combinations.append((name,year, month, day, hour, location_id))
        elif year > start_year:
            if year < end_year:
                days_in_month = calendar.monthrange(year, month)[1]
                days = range(1, days_in_month + 1)
                hours = range(24)
                for day, hour in itertools.product(days, hours):
                    combinations.append((name,year, month, day, hour, location_id))
            elif year == end_year:
                if month <= end_month:
                    days_in_month = calendar.monthrange(year, month)[1]
                    days = range(1, days_in_month + 1)
                    hours = range(24)
                    for day, hour in itertools.product(days, hours):
                        combinations.append((name,year, month, day, hour, location_id))


# 创建数据框
columns = ["Name","year", "month", "day", "hour", "PULocationID"]
df = pd.DataFrame(combinations, columns=columns)

# 記錄結束時間
end_time = time.time()   
# 計算程式執行花費的時間
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())

df

地點數量: 261

程式執行花費的時間： 8.91 秒
done時間: 2023-09-14 17:22:21.448667


,Name,year,month,day,hour,PULocationID
0,1,2023,9,1,0,3
1,1,2023,9,1,1,3
2,1,2023,9,1,2,3
3,1,2023,9,1,3,3
4,1,2023,9,1,4,3
...,...,...,...,...,...,...
2292619,3,2023,12,31,19,199
2292620,3,2023,12,31,20,199
2292621,3,2023,12,31,21,199
2292622,3,2023,12,31,22,199


In [4]:
#節日資訊
import numpy as np
from holidays.countries import US
from holidays.calendars import gregorian
from holidays.calendars.gregorian import _get_nth_weekday_of_month
class AllHolidays(US):
    def _populate(self, year):
        super()._populate(year)
        self.pop_named("Lincoln's Birthday")
        self.pop_named("Susan B. Anthony Day")
        
        self._add_holiday("Valentine's Day", 2, 14)
        self._add_holiday("St.Patrick's Day", 3, 17)
        self._add_holiday("Halloween", 10, 31)
     
        self._add_holiday("Christmas Eve", 12, 24)
        self._add_holiday("Christmas season", 12, 26)
        self._add_holiday("Christmas season", 12, 27)
        self._add_holiday("Christmas season", 12, 28)
        self._add_holiday("Christmas season", 12, 29)
        self._add_holiday("Christmas season", 12, 30)
        
        self._add_holiday("New Year's Eve", 12, 31)

# 實例化AllHolidays
ny_holidays_all = AllHolidays(subdiv='NY', years=[2019, 2020, 2021, 2022, 2023])

# 新增2019~2023年的復活節、復活節星期一
ny_holidays_all[datetime.date(2019, 4, 21)] = "Easter"
ny_holidays_all[datetime.date(2019, 4, 22)] = "Easter Monday"
ny_holidays_all[datetime.date(2020, 4, 12)] = "Easter"
ny_holidays_all[datetime.date(2020, 4, 13)] = "Easter Monday"
ny_holidays_all[datetime.date(2021, 4, 4)] = "Easter"
ny_holidays_all[datetime.date(2021, 4, 5)] = "Easter Monday"
ny_holidays_all[datetime.date(2022, 4, 17)] = "Easter"
ny_holidays_all[datetime.date(2022, 4, 18)] = "Easter Monday"
ny_holidays_all[datetime.date(2023, 4, 9)] = "Easter"
ny_holidays_all[datetime.date(2023, 4, 10)] = "Easter Monday"

# 新增2019~2023感恩節
ny_holidays_all[datetime.date(2019,11, 28)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2019,11, 29)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2019,11, 30)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 26)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 27)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 28)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 25)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 26)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 27)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 24)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 25)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 26)] =  "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 23)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 24)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 25)] = "Thanksgiving weekend"

#地點經緯度
datasite_temp = '../taxi_zone_lat_lon/organized_taxi_zone_lat_lon.csv'
#創立一個 df_temp 讀經緯度資料
df_temp = pd.read_csv(datasite_temp)

df_temp

,zone,LocationID,borough,lat,lon
0,Newark Airport,1,EWR,40.690243,-74.174270
1,Jamaica Bay,2,Queens,40.612163,-73.817643
2,Allerton/Pelham Gardens,3,Bronx,40.864294,-73.846510
3,Alphabet City,4,Manhattan,40.723853,-73.975209
4,Arden Heights,5,Staten Island,40.556678,-74.189803
...,...,...,...,...,...
258,Williamsburg (South Side),256,Brooklyn,40.710736,-73.962940
259,Woodlawn/Wakefield,259,Bronx,40.900107,-73.853635
260,Woodside,260,Queens,40.746439,-73.905907
261,World Trade Center,261,Manhattan,40.707456,-74.013983


In [5]:
# 記錄開始時間
start_time = time.time()

# 臨時日期
df['TempTime'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str)+'-' +df['day'].astype(str),format='%Y-%m-%d')
# weekday
df['weekday'] = (pd.to_datetime(df['TempTime'])).dt.weekday
# is_holiday
#df['is_holiday'] = np.where(df['TempTime'].dt.date.isin(ny_holidays_all), True, False)
#df['is_holiday'].value_counts()
#經緯度
df = df.merge(df_temp[['LocationID', 'lat','lon']], left_on=['PULocationID'], right_on=['LocationID'], how='left')
df = df.drop(['LocationID'],axis = 1)
del df_temp 

#留下需求月份
df = df[(df['TempTime'] >= output_start_day) &  (df['TempTime'] < output_end_day) ]
# 編號
df = df.sort_index(ignore_index=True)

# TempTime移除
df = df.drop(columns=['TempTime'])


# 輸出檔案
if df_save > 0 :
    os.makedirs(os.path.dirname(f'{custom_output_path}/{DataName}.parquet'), exist_ok=True)
    df.to_parquet(f'{custom_output_path}/{DataName}.parquet')
    print('save!')
    
# 記錄結束時間
end_time = time.time()   
# 計算程式執行花費的時間
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())

save!

程式執行花費的時間： 3.36 秒
done時間: 2023-09-14 17:22:25.071710


In [6]:
df

,Name,year,month,day,hour,PULocationID,weekday,lat,lon
0,1,2023,9,20,0,3,2,40.864294,-73.846510
1,1,2023,9,20,1,3,2,40.864294,-73.846510
2,1,2023,9,20,2,3,2,40.864294,-73.846510
3,1,2023,9,20,3,3,2,40.864294,-73.846510
4,1,2023,9,20,4,3,2,40.864294,-73.846510
...,...,...,...,...,...,...,...,...,...
1935571,3,2023,12,31,19,199,6,40.793489,-73.880683
1935572,3,2023,12,31,20,199,6,40.793489,-73.880683
1935573,3,2023,12,31,21,199,6,40.793489,-73.880683
1935574,3,2023,12,31,22,199,6,40.793489,-73.880683


---

TS11

In [10]:
import random
df['count'] = [random.randint(0, 10) for _ in range(len(df))]

In [11]:
df

,Name,year,month,day,hour,PULocationID,weekday,lat,lon,count
0,1,2023,9,20,0,3,2,40.864294,-73.846510,0
1,1,2023,9,20,1,3,2,40.864294,-73.846510,8
2,1,2023,9,20,2,3,2,40.864294,-73.846510,5
3,1,2023,9,20,3,3,2,40.864294,-73.846510,5
4,1,2023,9,20,4,3,2,40.864294,-73.846510,4
...,...,...,...,...,...,...,...,...,...,...
1935571,3,2023,12,31,19,199,6,40.793489,-73.880683,3
1935572,3,2023,12,31,20,199,6,40.793489,-73.880683,6
1935573,3,2023,12,31,21,199,6,40.793489,-73.880683,10
1935574,3,2023,12,31,22,199,6,40.793489,-73.880683,0


In [12]:
# 輸出檔案
if df_save > 0 :
    os.makedirs(os.path.dirname(f'{custom_output_path}/{DataName}.parquet'), exist_ok=True)
    df.to_parquet(f'{custom_output_path}/{DataName}.parquet')
    print('save!')
    

save!
